In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
#Load the trained model,scaler pickle file,onehotencoder pickle file
model = load_model('churn_model.h5')


#Load the encoder and scaler
with open('onehotencoder_geo.pkl', 'rb') as file:
    onehotencoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:   
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [3]:
#Example input data
input_data = {'CreditScore': 600,
              'Geography': 'France',
              'Gender': 'Male',
              'Age': 40,
                'Tenure': 3,
                'Balance': 60000,
                'NumOfProducts': 2,
                'HasCrCard': 1,
                'IsActiveMember': 1,
                'EstimatedSalary': 50000
                }

In [10]:
#one hot encoding for Geography
geo_encoded = onehotencoder_geo.transform(np.array(input_data['Geography']).reshape(-1,1)).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehotencoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

d:\Anaconda\envs\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [11]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [12]:
#Encode categorical data
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [13]:
#Concatenate the onehotencoded data
input_df = pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [14]:
#Scaling the input data using the scaler
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.52358424,  0.91739884,  0.1073061 , -0.69146874, -0.26845961,
         0.79740451,  0.64376017,  0.97067965, -0.87572587,  0.99401789,
        -0.57773517, -0.57234647]])

In [18]:
#Predict the output whether the person will churn or not(churn-leave the bank)
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 128ms/step


array([[0.04863323]], dtype=float32)

In [19]:
prediction = (prediction > 0.5)
prediction

array([[False]])

In [20]:
prediction_probability= prediction[0][0]
prediction_probability
if prediction_probability>0.5:
    print("The customer will leave the bank")
else:
    print("The customer will not leave the bank")

The customer will not leave the bank
